# Parameters

In [1]:
model_file          = "../models/europe.json"
load_series_file    = "data/loads.csv"
noise_series_file   = "data/cost_noise.csv"
nuclear_series_file = "data/nuclear_2016.csv"
border_flows_file   = "data/border_flows_2016.csv"
output_dir          = "data/TOPF_run";

# Network

In [2]:
using TemperateOptimalPowerFlow
using CSV, DataFrames, DataDrop

Import the model:

In [3]:
network = import_model(model_file)

Dict{String, Any} with 12 entries:
  "gen_nd"   => Dict{String, Any}()
  "bus"      => Dict{String, Any}("4304"=>Dict{String, Any}("coord"=>Any[11.843…
  "name"     => "europe"
  "dcline"   => Dict{String, Any}()
  "gen"      => Dict{String, Any}("1"=>Dict{String, Any}("entsoe_names"=>Any[],…
  "branch"   => Dict{String, Any}("4304"=>Dict{String, Any}("br_r"=>0.00174284,…
  "storage"  => Dict{String, Any}()
  "switch"   => Dict{String, Any}()
  "baseMVA"  => 100
  "per_unit" => true
  "shunt"    => Dict{String, Any}()
  "load"     => Dict{String, Any}("4304"=>Dict{String, Any}("load_bus"=>4304, "…

# Assign ramp constraints

Add ramp constraint by defining a maximum ramp rate of 200 MW / hour for coal generators 

In [4]:
for gen ∈ values(network["gen"])
    gen["max_ramp_rate"] = gen["aggregated_type"] == "coal" ? 2.0 : 0.0
end

# Nuclear production

Import actual time series for nuclear generators:

In [5]:
nuc_series_df = CSV.read(nuclear_series_file, DataFrame)
sort!(nuc_series_df, :id)
nuc_series_df.id .= string.(nuc_series_df.id)
nuc_series_df

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,79,1970.42,1970.52,1970.62,1970.62,1970.22,1968.94,1969.83,1969.93,1970.42,1969.63,1967.76,1965.79,1964.41,1963.04,1962.25,1962.45,1963.13,1960.67,1962.94,1963.04,1964.02,1964.51,1966.97,1966.97,1966.78,1964.41,1964.61,1964.71,1964.61,1963.43,1962.64,1962.94,1962.74,1963.04,1961.95,1960.58,1961.07,1961.07,1961.17,1961.17,1960.87,1961.36,1963.04,1965.5,1969.34,1969.83,1969.53,1970.42,1969.24,1967.76,1968.15,1967.47,1967.17,1967.07,1966.88,1967.07,1967.37,1967.17,1964.61,1961.95,1961.76,1959.79,1959.59,1958.41,1958.31,1959.79,1961.36,1961.56,1962.54,1963.13,1964.22,1960.08,1956.05,1957.23,1957.43,1956.44,1953.68,1953.29,1952.6,1956.15,1967.37,1968.25,1970.22,1964.02,1960.08,1958.61,1963.43,1956.05,1945.32,1943.35,1957.33,1956.15,1957.52,1957.92,1958.21,1956.44,1956.15,1954.08,1955.26,⋯
2,87,1062.4,1062.3,1062.9,1062.9,1063.6,1063.1,1063.1,1063.3,1062.9,1059.7,1061.1,1061.1,1061.1,1061.6,1062.1,1062.5,1062.6,1062.9,1063.3,1062.5,1062.8,1063.1,1062.8,1063.5,1064.1,1063.7,1063.8,1064.1,1065.0,1064.1,1064.8,1064.5,1065.1,1065.0,1064.9,1064.8,1065.1,1064.7,1065.4,1065.1,1064.8,1065.2,1064.9,1065.2,1064.8,1065.2,1065.2,1064.9,1064.6,1064.6,1065.5,1065.7,1065.9,1065.3,1065.2,1064.7,1065.6,1065.0,1065.6,1065.2,1065.2,1064.9,1065.1,1065.1,1064.6,1064.9,1064.5,1064.1,1063.6,1063.9,1063.9,1063.3,1064.5,1063.6,1063.6,1062.9,1064.1,1063.5,1063.4,1063.6,1063.9,1063.7,1063.4,1062.6,1063.6,1063.1,1063.7,1063.2,1063.0,1063.1,1063.1,1062.7,1062.8,1063.7,1062.9,1063.6,1062.9,1064.2,1064.6,⋯
3,160,975.114,975.404,975.791,975.888,975.694,975.501,975.888,975.211,975.501,975.791,973.759,975.501,976.081,976.275,975.694,976.468,977.339,976.855,977.146,975.888,975.694,975.694,975.501,976.081,975.888,976.178,975.985,976.178,975.888,975.985,975.791,975.404,975.211,977.146,978.21,975.404,975.114,975.501,975.694,975.694,975.888,975.888,975.985,976.372,975.985,976.178,976.662,976.275,975.888,976.662,977.049,976.275,976.081,976.855,976.468,976.662,976.275,976.275,976.565,976.952,977.339,976.081,975.888,975.985,976.081,975.791,976.372,975.404,976.081,976.275,976.275,976.372,976.081,977.049,976.468,976.468,977.533,976.468,976.952,978.113,978.79,978.307,978.5,978.5,978.887,979.371,978.5,978.984,978.79,978.887,978.79,978.693,978.79,978.5,978.597,978.693,978.5,978.887,978.887,⋯
4,176,929.4,927.9,929.1,929.3,928.8,929.5,930.1,929.9,929.4,929.6,929.1,929.5,930.3,930.4,930.3,930.8,930.2,930.1,930.9,930.7,930.3,930.5,930.2,930.1,929.7,929.8,928.9,928.8,929.2,927.4,927.5,927.8,926.5,927.6,927.8,927.0,926.9,926.7,926.9,926.7,927.2,927.2,926.9,927.3,926.0,926.3,926.3,925.7,925.2,925.1,924.9,924.0,923.8,923.6,922.4,924.5,925.1,924.3,925.1,925.3,924.7,924.1,924.2,923.8,922.9,923.3,923.1,922.1,923.4,924.1,923.6,922.6,921.7,920.0,919.9,920.3,919.3,919.3,919.3,918.6,919.2,919.4,919.4,918.2,918.8,919.2,918.6,919.2,920.2,919.6,919.9,920.2,919.6,921.9,922.3,920.5,921.0,920.8,918.9,⋯
5,179,1914.31,1914

Separate nuclear generators as non-dispatchable:

In [6]:
split_nondispatchable!(network, nuc_series_df.id)

Export the time series:

In [7]:
filter!(row -> string(row.id) in keys(network["gen_nd"]), nuc_series_df);

In [8]:
nuc_ids = nuc_series_df.id

44-element Vector{String}:
 "79"
 "87"
 "160"
 "176"
 "179"
 "209"
 "210"
 "232"
 "249"
 "256"
 "257"
 "265"
 "318"
 ⋮
 "723"
 "732"
 "751"
 "757"
 "782"
 "846"
 "908"
 "917"
 "921"
 "943"
 "950"
 "982"

In [9]:
nuc_series = Matrix(nuc_series_df[:, 2:end]) / 100.

44×8736 Matrix{Float64}:
 19.7042   19.7052   19.7062   19.7062   …  19.7042   19.7052   19.7052
 10.624    10.623    10.629    10.629       10.633    10.63     10.624
  9.75114   9.75404   9.75791   9.75888     10.4758    9.80338   9.75114
  9.294     9.279     9.291     9.293        9.299     9.296     9.296
 19.1431   19.1431   19.1431   19.1431      19.1431   19.1431   19.1431
 12.7429   12.736    12.477    12.7685   …  12.7764   12.7321   12.7311
 12.0824   12.0824   12.0787   12.0787      12.1026   12.1024   12.0844
 13.64     13.6173   13.553    13.53        13.6965   13.6178   13.635
 12.7271   12.7467   12.7437   12.7301      12.9856   12.9398   12.9096
 12.0348   12.0338   12.0309   12.023       12.0309   12.0328   12.0348
 12.8447   12.8466   12.8437   12.8437   …  12.8234   12.8251   12.8417
 14.0004   13.997    13.8992   13.7054      14.0317   14.0294   14.0088
 17.21     17.21     17.21     17.21        17.26     17.26     17.21
  ⋮                                      ⋱ 

# Loads

List of all countries in the network:

In [10]:
countries = Set([bus["country"] for bus ∈ values(network["bus"])])

Set{String} with 26 elements:
  "SI"
  "RO"
  "IT"
  "NL"
  "FR"
  "DK"
  "SK"
  "RS"
  "BG"
  "AT"
  "CH"
  "XX"
  "CZ"
  "ME"
  "DE"
  "MK"
  "BA"
  "BE"
  "AL"
  "LU"
  "HR"
  "ES"
  "GR"
  "HU"
  "PL"
  "PT"

### Compute expected production by country

Determine the total expected production:

In [11]:
expected_prod = Dict(country => 0.0 for country ∈ countries)

for gen ∈ values(network["gen"])
    expected_prod[gen["country"]] += gen["pexp"]
end

sort(expected_prod, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 277.804
  "PL" => 135.462
  "FR" => 128.144
  "IT" => 126.413
  "ES" => 77.2725
  "NL" => 59.6867
  "AT" => 57.6037
  "RO" => 48.9335
  "GR" => 48.2076
  "PT" => 42.3998
  "CZ" => 35.3887
  "RS" => 29.6354
  "BG" => 19.8427
  "BE" => 17.3597
  "BA" => 13.4466
  "CH" => 11.6612
  "HU" => 10.6986
  "SI" => 10.1473
  "SK" => 9.12782
  "DK" => 7.71758
  "HR" => 6.54311
  "MK" => 5.00832
  "AL" => 4.43366
  "LU" => 3.56473
  "XX" => 2.6395
  "ME" => 2.33425

### Compute import/export balance by country

Export balance by country:

In [12]:
border_flows_df = CSV.read(border_flows_file, DataFrame);

In [13]:
border_flows = Dict{String, Float64}()
for (country, flow) in eachrow(border_flows_df)
    border_flows[country] = flow / 100.0
end
border_flows["XX"] = 0.0

sort(border_flows, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 56.6309
  "FR" => 30.1593
  "CZ" => 12.5109
  "RO" => 6.64271
  "PT" => 5.73057
  "BA" => 4.25852
  "PL" => 2.34598
  "BG" => 1.89706
  "RS" => 1.67495
  "SI" => 1.27665
  "AL" => 0.0469134
  "LU" => 0.0
  "XX" => 0.0
  "ME" => -0.309394
  "MK" => -1.36593
  "DK" => -2.81724
  "CH" => -5.13167
  "SK" => -5.37435
  "HR" => -7.17674
  "BE" => -7.25616
  "GR" => -8.38059
  "HU" => -9.63747
  "NL" => -9.91312
  "AT" => -9.93982
  "ES" => -14.5411
  "IT" => -41.3309

Check: total out flow adds up to zero

In [14]:
sum(values(border_flows))

4.440892098500626e-16

### Compute nuclear production by country

Nuclear production by country:

In [15]:
avg_nuclear = sum(nuc_series, dims=2) / size(nuc_series, 2)

nuclear_prod = Dict(country => 0.0 for country ∈ countries)
for i = 1:length(nuc_ids)
    country = network["gen_nd"][nuc_ids[i]]["country"]
    nuclear_prod[country] += avg_nuclear[i]
end
nuclear_prod

Dict{String, Float64} with 26 entries:
  "SI" => 0.0
  "ES" => 45.3376
  "GR" => 0.0
  "LU" => 0.0
  "HU" => 16.8633
  "CH" => 22.9463
  "MK" => 0.0
  "RS" => 0.0
  "XX" => 0.0
  "DE" => 90.667
  "AL" => 0.0
  "FR" => 441.027
  "PL" => 0.0
  "IT" => 0.0
  "AT" => 0.0
  "HR" => 0.0
  "PT" => 0.0
  "ME" => 0.0
  "NL" => 4.22129
  "BE" => 46.9143
  "RO" => 11.829
  "BG" => 17.1781
  "CZ" => 24.8885
  "SK" => 16.7945
  "DK" => 0.0
  "BA" => 0.0

### Compute total load by country

Determine annual load by country:

In [16]:
total_load = Dict(country => prod + nuclear_prod[country] - border_flows[country]
    for (country, prod) ∈ expected_prod)

sort(total_load, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "FR" => 539.011
  "DE" => 311.84
  "IT" => 167.744
  "ES" => 137.151
  "PL" => 133.116
  "NL" => 73.8211
  "BE" => 71.5302
  "AT" => 67.5436
  "GR" => 56.5882
  "RO" => 54.1199
  "CZ" => 47.7663
  "CH" => 39.7391
  "HU" => 37.1993
  "PT" => 36.6692
  "BG" => 35.1237
  "SK" => 31.2967
  "RS" => 27.9604
  "HR" => 13.7198
  "DK" => 10.5348
  "BA" => 9.18812
  "SI" => 8.87068
  "MK" => 6.37425
  "AL" => 4.38674
  "LU" => 3.56473
  "ME" => 2.64364
  "XX" => 2.6395

### Import loads

In [17]:
normalized_loads_df = CSV.read(load_series_file, DataFrame)
sort!(normalized_loads_df, :id)
normalized_loads_df.id .= string.(normalized_loads_df.id)
normalized_loads_df

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,81,0.938493,0.94056,0.94431,0.965798,0.995459,1.05127,1.16325,1.26474,1.29085,1.25235,1.21141,1.17407,1.1521,1.15649,1.23743,1.37643,1.42806,1.42263,1.37529,1.31057,1.25499,1.16635,1.07034,1.0009,0.977794,0.967649,0.987112,1.03538,1.16843,1.30371,1.40086,1.4401,1.39005,1.33994,1.26785,1.23734,1.22261,1.23272,1.26538,1.36408,1.43871,1.43944,1.41347,1.34664,1.27334,1.16861,1.0984,1.04564,1.011,1.00121,1.01443,1.04507,1.15891,1.26777,1.3475,1.37358,1.31931,1.30955,1.27279,1.26137,1.26058,1.27036,1.30045,1.35307,1.40458,1.38262,1.33534,1.26476,1.24146,1.14755,1.05549,1.00036,0.961356,0.943444,0.952851,0.989175,1.07358,1.17976,1.26521,1.28799,1.22944,1.18868,1.14799,1.13353,1.1277,1.14103,1.17177,1.27355,1.34241,1.31568,1.27879,1.22745,1.20805,1.14693,1.00462,0.954359,0.927759,0.900434,0.899021,⋯
2,82,0.830213,0.812621,0.818705,0.849257,0.934548,1.02095,1.12207,1.18907,1.187,1.17177,1.13522,1.1127,1.0988,1.1013,1.15338,1.25454,1.30547,1.32635,1.26768,1.21336,1.15094,1.03888,0.920702,0.853106,0.82458,0.820521,0.836946,0.881376,1.02677,1.16136,1.24447,1.26348,1.2206,1.18775,1.13147,1.1133,1.10367,1.12093,1.15171,1.25553,1.33124,1.32972,1.30593,1.24577,1.1982,1.1125,1.0216,0.957921,0.940011,0.926531,0.941158,0.970809,1.11191,1.24624,1.31123,1.32745,1.26927,1.23486,1.17871,1.16353,1.1483,1.16959,1.22602,1.32309,1.40239,1.38722,1.33316,1.27546,1.24017,1.15028,1.05164,0.994951,0.95755,0.939886,0.958975,0.99893,1.11555,1.25753,1.31596,1.31289,1.23259,1.16594,1.10069,1.08373,1.04972,1.05235,1.09902,1.25209,1.36699,1.34941,1.3243,1.27192,1.23523,1.14475,1.00233,0.966186,0.935197,0.935968,0.94981,⋯
3,83,1.13311,1.11646,1.11714,1.15898,1.30032,1.43719,1.52174,1.57789,1.57978,1.57263,1.54614,1.54237,1.53566,1.54174,1.56996,1.63365,1.69739,1.70322,1.62485,1.56718,1.52987,1.41797,1.29484,1.21598,1.17073,1.15488,1.16725,1.203,1.32861,1.45047,1.52421,1.55351,1.54578,1.52626,1.48756,1.46674,1.46743,1.48499,1.50224,1.58992,1.66201,1.66409,1.61976,1.546,1.50301,1.41361,1.29325,1.20545,1.16485,1.15143,1.15868,1.18803,1.31221,1.43777,1.503,1.55728,1.54806,1.54313,1.52006,1.52486,1.53058,1.54802,1.57025,1.62627,1.66584,1.64378,1.58072,1.5168,1.48587,1.39312,1.27858,1.21159,1.16545,1.14125,1.14083,1.16974,1.27635,1.40848,1.48475,1.54393,1.52353,1.52243,1.49945,1.49468,1.4764,1.49392,1.50816,1.58745,1.65694,1.63623,1.5773,1.52007,1.47581,1.38671,1.26564,1.19322,1.15532,1.14512,1.15278,⋯
4,84,0.997376,0.999261,1.00261,1.03684,1.12962,1.23691,1.34919,1.44279,1.46622,1.44934,1.42175,1.40935,1.39306,1.38904,1.43287,1.51018,1.54617,1.54406,1.47882,1.4065,1.34481,1.24003,1.12478,1.05454,1.02776,1.01664,1.0344,1.07828,1.21465,1.34989,1.43389,1.47295,1.43459,1.38966,1.32406,1.29912,1.28097,1.29044,1.30969,1.41179,1.49206,1.48607,1.46191,1.39217,1.32998,1.22723,1.14153,1.07661,1.0409,1.0279,1.03738,1.06952,1.19762,1.31327,1.39343,1.42417,1.36776,1.34832,1.29925,1.28764,1.28645,1.29026,

Check that the IDs match:

In [18]:
bus_ids = normalized_loads_df.id
issetequal(bus_ids, keys(network["bus"]))

true

In [19]:
normalized_loads = Matrix(normalized_loads_df[:, 2:end])

4097×8736 Matrix{Float64}:
 0.938493  0.94056   0.94431   0.965798  …  1.05357   1.00106   0.947791
 0.830213  0.812621  0.818705  0.849257     0.998426  0.913916  0.862687
 1.13311   1.11646   1.11714   1.15898      1.35473   1.23778   1.17104
 0.997376  0.999261  1.00261   1.03684      1.15782   1.07983   1.01723
 0.985948  0.977606  0.978128  0.999351     1.16584   1.08094   1.02337
 0.98941   1.00199   1.02122   1.0892    …  1.13185   1.04611   0.989368
 1.13717   1.13256   1.14522   1.17777      1.3057    1.2147    1.15431
 1.04948   1.0749    1.09954   1.18281      1.17858   1.09837   1.03958
 0.882988  0.869205  0.866138  0.881341     1.0401    0.968909  0.917465
 0.73698   0.733618  0.74721   0.772693     0.841092  0.786889  0.745047
 0.95939   0.936714  0.906511  0.906641  …  1.17119   1.08984   1.02842
 1.41993   1.40153   1.39603   1.41262      1.64467   1.53326   1.45859
 0.925378  0.910568  0.902068  0.917818     1.09964   1.02309   0.968801
 ⋮                             

Check normalization:

In [20]:
sum(normalized_loads) / length(normalized_loads)

1.0001413983031382

Normalization by country:

In [21]:
normalized_annual_loads = Dict(country => [] for country ∈ countries)
for (id, annual_load) ∈ Dict(bus_ids .=> sum(normalized_loads, dims=2) / size(normalized_loads, 2))
    push!(normalized_annual_loads[network["bus"][id]["country"]], annual_load)
end

normalization = Dict(country => sum(annual_loads) / length(annual_loads)
    for (country, annual_loads) ∈ normalized_annual_loads)

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.00293
  "HU" => 0.999826
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.00069
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.00174
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.00025
  "BE" => 1.00043
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Check that the load assignement within each country is properly normalized:

In [22]:
load_prop_by_country = Dict(country => 0.0 for country in countries)
for bus ∈ values(network["bus"])
    load_prop_by_country[bus["country"]] += bus["load_prop"]
end

load_prop_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.0
  "HU" => 1.0
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.0
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.0
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.0
  "BE" => 1.0
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Compute a scaling value for each country:

In [23]:
load_scaling = [(
        network["bus"][id]["load_prop"]
        * normalization[network["bus"][id]["country"]]
        * total_load[network["bus"][id]["country"]]
        )
    for id ∈ bus_ids]

4097-element Vector{Float64}:
 0.6349757755932048
 0.544365227164444
 0.35161303893236123
 0.27118902295050934
 0.16097839463621658
 0.2955407291362091
 0.45843825167784835
 0.19874288379779137
 0.3974857674630873
 0.21553571730621504
 0.19145321975182375
 0.12211186908659864
 0.19913420268885207
 ⋮
 0.3286106478409917
 0.397822859038949
 0.08224945002111711
 0.46104298799054877
 0.28933968581938074
 0.5786793714458685
 0.6774609942543717
 0.013439436507376995
 0.024738050704946425
 0.18390510024021875
 0.038924549500323306
 0.07914678370329992

In [24]:
loads = load_scaling .* normalized_loads

4097×8736 Matrix{Float64}:
 0.59592    0.597233   0.599614   …  0.668993   0.635648   0.601824
 0.451939   0.442363   0.445674      0.543509   0.497504   0.469617
 0.398415   0.392563   0.392802      0.476339   0.435221   0.411753
 0.270477   0.270989   0.271896      0.313988   0.292839   0.275861
 0.158716   0.157374   0.157458      0.187674   0.174009   0.16474
 0.292411   0.296129   0.301812   …  0.334508   0.309167   0.292399
 0.521323   0.51921    0.525013      0.598584   0.556866   0.52918
 0.208577   0.213628   0.218526      0.234234   0.218293   0.206609
 0.350975   0.345497   0.344277      0.413426   0.385128   0.364679
 0.158846   0.158121   0.16105       0.181285   0.169603   0.160584
 0.183678   0.179337   0.173554   …  0.224227   0.208653   0.196894
 0.173391   0.171144   0.170471      0.200834   0.187229   0.178111
 0.184274   0.181325   0.179633      0.218975   0.203733   0.192921
 ⋮                                ⋱                        ⋮
 0.28682    0.278163   0.27894

Check that the annual load matches the expectations:

In [25]:
sum(loads) / size(loads, 2) / sum(values(total_load))

1.0004002959745075

# Generation costs

In [26]:
gen_costs = Matrix(CSV.read(noise_series_file, DataFrame))

771×8736 Matrix{Float64}:
 -0.0262487    0.0198198    0.0790532   …  -0.0666133    0.0327033
 -0.0707873   -0.126037    -0.21975        -0.225453    -0.282577
  0.105176     0.069375     0.194085        0.072822     0.022119
 -0.0222022   -0.0534002   -0.150654       -0.0145013   -0.087602
 -0.0312321   -0.071578    -0.0364149      -0.0838126   -0.0406127
 -0.0341725    0.0172165    0.110258    …   0.00594135  -0.0312075
 -0.165547    -0.0400537    0.0516328       0.0739772   -0.0328689
 -0.134217    -0.191215    -0.132414       -0.132683    -0.0660482
 -0.196525    -0.0747628    0.0353276      -0.125528    -0.164061
  0.132592     0.00516041   0.0292117       0.15272      0.025195
 -1.53015e-5   0.120748     0.00266818  …  -0.032617    -0.027211
 -0.125775    -0.105427    -0.0152052       0.0769087    0.0717783
  0.171401     0.163038     0.144398        0.297562     0.0635374
  ⋮                                     ⋱                ⋮
 -0.00495856   0.0705164    0.0291467       0.0573

# Setup

In [27]:
setup(output_dir, network, loads, gen_costs, nuc_series)

[2024-07-25 10:01:33]  Checking balance: 1930.1422890820186 (production) / 1930.9149172705668 (consumption)
[2024-07-25 10:01:33]  Creating directory 'data/TOPF_run'
[2024-07-25 10:01:33]  Loading network
[2024-07-25 10:01:33]  Computing susceptance matrix
[2024-07-25 10:01:34]  Computing PTDF matrix (this may take some time)
[2024-07-25 10:02:30]  Distributing generator on buses
[2024-07-25 10:02:30]  Distributing loads on buses
[2024-07-25 10:02:30]  Distributing non-dispatchable generators on buses
[2024-07-25 10:02:31]  Computing max capacity for each generator
[2024-07-25 10:02:31]  Computing expected production for each generator
[2024-07-25 10:02:31]  Computing thermal limits
[2024-07-25 10:02:31]  Computing quadratic line costs
[2024-07-25 10:02:31]  Computing linear line costs
[2024-07-25 10:02:42]  Computing linear generation costs
[2024-07-25 10:02:42]  Computing total load constraints
[2024-07-25 10:02:42]  Computing ramp constraints
[2024-07-25 10:02:42]  Record generators

In [28]:
readdir(output_dir)

17-element Vector{String}:
 "A_gen.h5"
 "A_load.h5"
 "A_nondispatch.h5"
 "A_ramp.h5"
 "PTDF_matrix.h5"
 "P_exp.h5"
 "P_load.h5"
 "P_max.h5"
 "P_nondispatch.h5"
 "P_total.h5"
 "gen_ids.h5"
 "linear_gen_cost.h5"
 "linear_line_cost.h5"
 "quadratic_cost.h5"
 "ramp_max.h5"
 "susceptance.h5"
 "thermal_limits.h5"

# List bus names associated with generators

This can be used to analyse the output:

In [29]:
gen_names = [network["bus"][string(network["gen"][id]["gen_bus"])]["name"] for id in get_ordered_gen_ids(network)];